# 참고 자료
- https://medium.com/geekculture/predicting-customer-life-time-value-cltv-via-beta-geometric-negative-binominal-distribution-59be07ac30bd

# 모듈 및 데이터 불러오기

In [1]:
import datetime

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import time
import pandas as pd
import matplotlib.pyplot as plt

from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

from sklearn.preprocessing import MinMaxScaler

##### UCI 교육용 데이터를 활용 
https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

In [2]:
df = pd.read_excel('./data/online_retail_II.xlsx', sheet_name = 'Year 2009-2010')
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


# EDA

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [4]:
# 분석에 필요한 columns
# Invoice, InvoiceDate, 'Quantity', 'Price', 'Customer ID'

# df.Invoice.apply(int)
# Invoice value 값 내에 'C'가 들어가 있는 경우, Refund건

df = df[df.Invoice.apply(lambda x: True if 'C' not in str(x) else False)]
df.index = range(len(df))

In [5]:
# Customer ID가 없는 경우, 고객에 대한 분석 불가능하기 때문에 해당 record 삭제 

df = df[df['Customer ID'].notnull()]
df.index = range(len(df))

In [6]:
df.Quantity.describe()

count    407695.000000
mean         13.586686
std          96.842229
min           1.000000
25%           2.000000
50%           5.000000
75%          12.000000
max       19152.000000
Name: Quantity, dtype: float64

In [7]:
df.Price.describe()

count    407695.000000
mean          3.294188
std          34.756655
min           0.000000
25%           1.250000
50%           1.950000
75%           3.750000
max       10953.500000
Name: Price, dtype: float64

In [8]:
print('분석 단위 최소 날짜: ', df.InvoiceDate.min(), '\n',
      '분석 단위 최대 날짜: ', df.InvoiceDate.max())

분석 단위 최소 날짜:  2009-12-01 07:45:00 
 분석 단위 최대 날짜:  2010-12-09 20:01:00


# 데이터 분석

In [9]:
# 필요한 데이터 남기고, 필요한 변수 새로 생성 

df = df[['Invoice', 'InvoiceDate', 'Quantity', 'Price', 'Customer ID']]
df['TotalPrice'] = df['Quantity']*df['Price']

df.head(3)

,Invoice,InvoiceDate,Quantity,Price,Customer ID,TotalPrice
0,489434,2009-12-01 07:45:00,12,6.95,13085.0,83.4
1,489434,2009-12-01 07:45:00,12,6.75,13085.0,81.0
2,489434,2009-12-01 07:45:00,12,6.75,13085.0,81.0


## RFM anlaysis 

- Recency
- Frequency
- Monetary

In [10]:
today_date = datetime(2010, 12, 11)

# 데이터의 날짜 시점과 현재 분석 기간의 차이가 멀어지면 유의미한 결과를 얻기 어려움 

In [11]:
# RFM 계산
rfm = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max()-date.min()).days, # recency
                                                    lambda date: (today_date-date.min()).days], # tenure 
                                     'Invoice': lambda num: num.nunique(),                       # frequency
                                     'TotalPrice' : lambda price: price.sum()})                 # monetary
rfm.head(2)

InvoiceDate             Invoice TotalPrice
             <lambda_0> <lambda_1> <lambda>   <lambda>
Customer ID                                           
12346.0             196        361       11     372.86
12347.0              37         40        2    1323.32

In [12]:
rfm.columns = rfm.columns.droplevel(0)
rfm.columns = ['recency', 'T', 'frequency', 'monetary']

In [13]:
# 평균 monetary - CLTV에서 한번 구매시 어느정도의 가격을 지불하는지 알기 위해 조정
rfm['monetary_avg'] = rfm['monetary']/rfm['frequency']

In [14]:
# recency와 T(첫 구매부터 오늘까지의 기간)을 주간 단위로 변경 
# lifetimes의 Params 조건에 맞추기 위해

rfm["recency_weekly"] = rfm["recency"] / 7
rfm["T_weekly"] = rfm["T"] / 7

In [15]:
# 2회 이상 구매한 고객(재구매 고객)에 대한 예측
rfm = rfm[rfm["frequency"] > 1] 

In [16]:
# rfm table 
rfm

,recency,T,frequency,monetary,monetary_avg,recency_weekly,T_weekly
Customer ID,,,,,,,
12346.0,196,361,11,372.86,33.896364,28.000000,51.571429
12347.0,37,40,2,1323.32,661.660000,5.285714,5.714286
12349.0,181,225,3,2671.14,890.380000,25.857143,32.142857
12352.0,16,28,2,343.80,171.900000,2.285714,4.000000
12356.0,44,60,3,3562.25,1187.416667,6.285714,8.571429
...,...,...,...,...,...,...,...
18276.0,336,365,5,1320.66,264.132000,48.000000,52.142857
18277.0,97,130,4,1069.67,267.417500,13.857143,18.571429
18283.0,275,294,6,641.77,106.961667,39.285714,42.000000


## 예상되는 구매 횟수 - BG/NBD Model

In [17]:
rfm[rfm["frequency"] == 0]

,recency,T,frequency,monetary,monetary_avg,recency_weekly,T_weekly
Customer ID,,,,,,,


In [18]:
# 모델 수렴을 위한 0값 조정 
rfm.loc[rfm["recency_weekly"] == 0, 'recency_weekly'] = 0.00000001

In [19]:
rfm.loc[rfm["T_weekly"] == 0, 'recency_weekly']

Series([], Name: recency_weekly, dtype: float64)

In [20]:
# 고객의 구매 횟수를 예측함

# 모델 생성 - 통상적으로 penalizer_coef = 0.001 to 0.1 
bgf = BetaGeoFitter(penalizer_coef=0.001)  

# 모델 학습
bgf.fit(rfm['frequency'],                 
        rfm['recency_weekly'],
        rfm['T_weekly'])

<lifetimes.BetaGeoFitter: fitted with 2893 subjects, a: 0.14, alpha: 10.85, b: 2.72, r: 2.13>

In [21]:
rfm["expected_purc_6_month"] = bgf.predict(6*4, # 주 기준 
                                           rfm['frequency'],
                                           rfm['recency_weekly'],
                                           rfm['T_weekly'])

In [22]:
# 다음 6개월 간 구매 횟수가 가장 많은 상위 10명의 고객 

rfm.sort_values('expected_purc_6_month', ascending=False).head(10)

,recency,T,frequency,monetary,monetary_avg,recency_weekly,T_weekly,expected_purc_6_month
Customer ID,,,,,,,,
14911.0,373,374,205,152147.57,742.183268,53.285714,53.428571,75.444341
17850.0,362,370,155,51208.87,330.379806,51.714286,52.857143,56.907868
12748.0,369,371,144,22879.66,158.886528,52.714286,53.000000,53.532914
15311.0,373,374,121,56003.26,462.836860,53.285714,53.428571,44.828553
13089.0,368,373,109,57912.03,531.303028,52.571429,53.285714,40.428996
14606.0,369,372,102,18704.41,183.376569,52.714286,53.142857,38.029555
14156.0,366,374,102,196566.74,1927.124902,52.285714,53.428571,37.633939
13694.0,361,371,94,131443.19,1398.331809,51.571429,53.000000,34.825384
17841.0,372,373,91,29892.50,328.489011,53.142857,53.285714,33.967579


In [23]:
# 다음 6개월 동안 예상되는 전체 구매 횟수

rfm["expected_purc_6_month"].sum()

10443.400529362489

## 예상 되는 구매 가격 - GAMMA-GAMMA Model

In [24]:
# 모델 생성
ggf = GammaGammaFitter(penalizer_coef=0.01) 
# 모델 학습
ggf.fit(rfm['frequency'], rfm['monetary_avg']) 

<lifetimes.GammaGammaFitter: fitted with 2893 subjects, p: 3.79, q: 0.34, v: 3.72>

In [25]:
# 고객별 구매액 예측
rfm["expected_average_profit"] = ggf.conditional_expected_average_profit(rfm['frequency'],
                                                                         rfm['monetary_avg'])

In [26]:
# 예상되는 구매 가격이 가장 높은 상위 10명의 고객 
rfm.sort_values("expected_average_profit", ascending=False).head(10)

,recency,T,frequency,monetary,monetary_avg,recency_weekly,T_weekly,expected_purc_6_month,expected_average_profit
Customer ID,,,,,,,,,
17450.0,70,74,7,52422.30,7488.900000,1.000000e+01,10.571429,9.430266,7681.447972
15760.0,0,266,2,13916.34,6958.170000,1.000000e-08,38.000000,0.097281,7627.355279
13902.0,92,360,5,34095.26,6819.052000,1.314286e+01,51.428571,0.133285,7067.119898
12357.0,0,24,2,12079.99,6039.995000,1.000000e-08,3.428571,5.706919,6621.146066
14091.0,44,334,2,9530.08,4765.040000,6.285714e+00,47.714286,0.230530,5223.948945
14088.0,14,57,3,13544.99,4514.996667,2.000000e+00,8.142857,4.950788,4795.973295
17940.0,291,372,6,26286.75,4381.125000,4.157143e+01,53.142857,2.722070,4513.371617
18102.0,373,374,89,349164.35,3923.194944,5.328571e+01,53.428571,33.164722,3930.965384
14028.0,0,186,3,10396.50,3465.500000,1.000000e-08,26.571429,0.172242,3681.470691


## CLTV 계산

In [27]:
# CLTV 계산
pre_cltv = ggf.customer_lifetime_value(bgf,
                                   rfm['frequency'],
                                   rfm['recency_weekly'],
                                   rfm['T_weekly'],
                                   rfm['monetary_avg'],
                                   time=6,   # 6개월
                                   freq="W", # 주 단위
                                   discount_rate=0.01)
pre_cltv

Customer ID
12346.0      26.888640
12347.0    4046.505116
12349.0    2567.818158
12352.0    1151.368946
12356.0    7405.385756
              ...     
18276.0     736.765972
18277.0    1305.266570
18283.0     404.888754
18286.0    1026.442166
18287.0    2173.814215
Name: clv, Length: 2893, dtype: float64

In [28]:
pre_cltv = pre_cltv.reset_index()

In [29]:
cltv_final = pd.merge(rfm, pre_cltv, on = 'Customer ID', how='left')
cltv_final

,Customer ID,recency,T,frequency,monetary,monetary_avg,recency_weekly,T_weekly,expected_purc_6_month,expected_average_profit,clv
0,12346.0,196,361,11,372.86,33.896364,28.000000,51.571429,0.737881,34.788047,26.888640
1,12347.0,37,40,2,1323.32,661.660000,5.285714,5.714286,5.325256,727.138685,4046.505116
2,12349.0,181,225,3,2671.14,890.380000,25.857143,32.142857,2.590491,946.847479,2567.818158
3,12352.0,16,28,2,343.80,171.900000,2.285714,4.000000,5.787292,190.420713,1151.368946
4,12356.0,44,60,3,3562.25,1187.416667,6.285714,8.571429,5.612020,1262.282629,7405.385756
...,...,...,...,...,...,...,...,...,...,...,...
2888,18276.0,336,365,5,1320.66,264.132000,48.000000,52.142857,2.562516,274.482131,736.765972
2889,18277.0,97,130,4,1069.67,267.417500,13.857143,18.571429,4.445793,280.623835,1305.266570
2890,18283.0,275,294,6,641.77,106.961667,39.285714,42.000000,3.488953,110.813603,404.888754
2891,18286.0,247,359,2,1296.43,648.215000,35.285714,51.285714,1.375558,712.404584,1026.442166


In [30]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(cltv_final[["clv"]])

cltv_final["scaled_clv"] = scaler.transform(cltv_final[["clv"]])

In [31]:
cltv_final["segment"] = pd.qcut(cltv_final["scaled_clv"], 5, labels=["about_to_sleep", "at_risk", "need_attention","loyal_customers", "champions"])

In [32]:
cltv_final.groupby("segment")["clv"].describe()

,count,mean,std,min,25%,50%,75%,max
segment,,,,,,,,
about_to_sleep,579.0,228.108481,134.267643,0.000004,112.218126,240.650795,346.601498,440.148439
at_risk,578.0,616.824789,100.372996,440.265172,532.007519,618.535041,706.103518,785.987491
need_attention,579.0,996.039497,127.513540,786.907390,888.224667,992.246232,1100.707898,1232.386434
loyal_customers,578.0,1576.382653,222.652665,1233.103601,1385.170928,1552.964789,1761.615793,2002.541343
champions,579.0,5202.697637,9468.972610,2003.459347,2355.305699,3044.558155,4516.679040,136569.313466


# 모델 검증

## 검증 데이터 불러오기 및 전처리

In [33]:
valdidation = pd.read_excel('./data/online_retail_II.xlsx', sheet_name = 'Year 2010-2011')
valdidation.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [34]:
# 분석에 필요한 columns
# Invoice, InvoiceDate, 'Quantity', 'Price', 'Customer ID'

valdidation = valdidation[valdidation.Invoice.apply(lambda x: True if 'C' not in str(x) else False)]
valdidation.index = range(len(valdidation))

In [35]:
# Customer ID가 없는 경우, 고객에 대한 분석 불가능하기 때문에 해당 record 삭제 

valdidation = valdidation[valdidation['Customer ID'].notnull()]
valdidation.index = range(len(valdidation))

In [36]:
print('분석 단위 최소 날짜: ', valdidation.InvoiceDate.min(), '\n',
      '분석 단위 최대 날짜: ', valdidation.InvoiceDate.max())

분석 단위 최소 날짜:  2010-12-01 08:26:00 
 분석 단위 최대 날짜:  2011-12-09 12:50:00


In [37]:
# 2010-12-10 ~ 2011-06-10 기간 분석 

valdidation = valdidation[(valdidation.InvoiceDate >= '2010-12-10')&\
                            (valdidation.InvoiceDate <= '2011-06-10')]

valdidation.index = range(len(valdidation))

In [38]:
# 필요한 데이터 남기고, 필요한 변수 새로 생성 

valdidation = valdidation[['Invoice', 'InvoiceDate', 'Quantity', 'Price', 'Customer ID']]
valdidation['TotalPrice'] = df['Quantity']*df['Price']

valdidation.head(3)

,Invoice,InvoiceDate,Quantity,Price,Customer ID,TotalPrice
0,538172,2010-12-10 09:33:00,12,1.25,15805.0,83.4
1,538172,2010-12-10 09:33:00,8,4.95,15805.0,81.0
2,538172,2010-12-10 09:33:00,12,2.55,15805.0,81.0


In [39]:
today_date = datetime(2011, 6, 11)

In [40]:
# RFM 계산
val_rfm = valdidation.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max()-date.min()).days, # recency
                                                                  lambda date: (today_date-date.min()).days], # tenure 
                                                  'Invoice': lambda num: num.nunique(),                       # frequency
                                                  'TotalPrice' : lambda price: price.sum()})                 # monetary
val_rfm.head(2)

InvoiceDate             Invoice TotalPrice
             <lambda_0> <lambda_1> <lambda>   <lambda>
Customer ID                                           
12346.0               0        143        1      11.80
12347.0             133        135        3    3164.58

In [41]:
val_rfm.columns = val_rfm.columns.droplevel(0)
val_rfm.columns = ['recency', 'T', 'frequency', 'monetary']

## 비교 테이블 만들기

In [42]:
compare_df = pd.merge(cltv_final[['Customer ID', 'expected_purc_6_month', 'clv']], 
                      val_rfm[['frequency', 'monetary']], 
                      how = 'left', 
                      on='Customer ID')
compare_df.columns = ['Customer ID', 'exp_6m_freq', 
                      'exp_cltv', 'real_freq', 'real_monetary']

compare_df

,Customer ID,exp_6m_freq,exp_cltv,real_freq,real_monetary
0,12346.0,0.737881,26.888640,1.0,11.80
1,12347.0,5.325256,4046.505116,3.0,3164.58
2,12349.0,2.590491,2567.818158,NaN,NaN
3,12352.0,5.787292,1151.368946,5.0,740.23
4,12356.0,5.612020,7405.385756,2.0,1255.28
...,...,...,...,...,...
2888,18276.0,2.562516,736.765972,NaN,NaN
2889,18277.0,4.445793,1305.266570,NaN,NaN
2890,18283.0,3.488953,404.888754,5.0,3389.89
2891,18286.0,1.375558,1026.442166,NaN,NaN


In [43]:
# 예측했으나 고객의 구매가 일어나지 않은 경우 0
compare_df.fillna(0, inplace=True) 

## 평가

In [44]:
from sklearn import metrics
import math

In [48]:
# RMSE - Frequency
metrics.mean_squared_error(compare_df['real_freq'], compare_df['exp_6m_freq'])**0.5

3.1171843166137574

In [49]:
# RMSE - Monetary
metrics.mean_squared_error(compare_df['real_monetary'], compare_df['exp_cltv'])**0.5

4494.344833396606

In [51]:
# 고객별 예측치와 실제데이터의 차이 계산
### + : 예측 > 실제 , - : 예측 < 실제
compare_df['gap_freq'] = compare_df['exp_6m_freq'] - compare_df['real_freq']
compare_df['gap_cltv'] = compare_df['exp_cltv'] - compare_df['real_monetary']
compare_df.head(5)

,Customer ID,exp_6m_freq,exp_cltv,real_freq,real_monetary,gap_freq,gap_cltv
0,12346.0,0.737881,26.888640,1.0,11.80,-0.262119,15.088640
1,12347.0,5.325256,4046.505116,3.0,3164.58,2.325256,881.925116
2,12349.0,2.590491,2567.818158,0.0,0.00,2.590491,2567.818158
3,12352.0,5.787292,1151.368946,5.0,740.23,0.787292,411.138946
4,12356.0,5.612020,7405.385756,2.0,1255.28,3.612020,6150.105756


In [61]:
# 예측만큼 구매가 이루어지지 않은 고객 조회
# CLTV의 허용 오차를 어느정도로 할 것인가 결정 필요!
# 허용 오차에서 벗어난 고객들 대상으로 '왜 구매가 적게 일어났는지?' 이해하는 과정 필요

chk = compare_df.sort_values('gap_cltv')
chk[(chk['gap_cltv']>0)]

,Customer ID,exp_6m_freq,exp_cltv,real_freq,real_monetary,gap_freq,gap_cltv
61,12482.0,5.224395e-09,0.000004,0.0,0.00,5.224395e-09,0.000004
2497,17448.0,1.067225e-06,0.000355,0.0,0.00,1.067225e-06,0.000355
1444,15359.0,3.508435e-04,0.008691,0.0,0.00,3.508435e-04,0.008691
468,13413.0,1.251466e-02,0.974686,0.0,0.00,1.251466e-02,0.974686
2751,17945.0,1.211787e-02,1.635643,0.0,0.00,1.211787e-02,1.635643
...,...,...,...,...,...,...,...
616,13694.0,3.482538e+01,51109.497652,26.0,6618.30,8.825384e+00,44491.197652
846,14156.0,3.763394e+01,76106.399479,18.0,10488.46,1.963394e+01,65617.939479
2499,17450.0,9.430266e+00,75742.703586,12.0,1786.24,-2.569734e+00,73956.463586
1096,14646.0,2.890395e+01,96642.048677,27.0,15262.93,1.903955e+00,81379.118677
